In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from pathlib import Path

2023-11-27 10:07:54.346349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 10:07:55.012831: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("val*.tfrecords"))

In [3]:
BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['age']
    return (ecg_data, example['gender']), label

@tf.function
def drop_na_ages(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(y))

@tf.function
def age_lt_90(x,y):
    return tf.math.reduce_all(tf.math.less_equal(y, tf.constant([90.0])))

@tf.function
def apply_random_noise(x,y):
    new_ecg = x[0] + tf.random.uniform(minval=-0.5, maxval=0.5, shape=(5000, 12))
    old_gender = x[1]
    return ((new_ecg,old_gender), y)

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(drop_na_ages)
    dataset = dataset.filter(age_lt_90)
    return dataset

def get_dataset(filenames, training=False):
    dataset = load_dataset(filenames)
    if training:
        dataset = dataset.map(apply_random_noise)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [4]:
train_dataset = get_dataset(TRAIN_RECS, training=True)
val_dataset = get_dataset(VAL_RECS)

2023-11-27 10:07:58.739329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 10:08:00.767299: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78812 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0


In [6]:
#tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [7]:
def residual_unit(x, y, n_samples_out, n_filters_out, prefix, kernel_size = 16):
    n_samples_in = y.shape[1]
    downsample = n_samples_in // n_samples_out
    n_filters_in = y.shape[2]
    
    if downsample == 1:
        y = y
    else:
        y = layers.MaxPooling1D(downsample, strides=downsample, padding='same', name = f"{prefix}_mp_opt")(y)
        
    if n_filters_in != n_filters_out:
        y = layers.Conv1D(n_filters_out, 1, padding='same', use_bias=False, name = f"{prefix}_conv_opt")(y)
        
    x = layers.Conv1D(n_filters_out, kernel_size, padding='same', use_bias=False, name = f"{prefix}_conv1")(x)
    x = layers.BatchNormalization(name = f"{prefix}_bn1")(x)
    x = layers.Activation("relu", name = f"{prefix}_act1")(x)
    x = layers.Dropout(0.2, name = f"{prefix}_dropout1")(x)
    x = layers.Conv1D(n_filters_out, kernel_size, strides=downsample, padding='same', use_bias=False, name = f"{prefix}_conv2")(x)

    x = layers.Add(name = f"{prefix}_add")([x,y])
    y = x
    x = layers.BatchNormalization(name = f"{prefix}_bn2")(x)
    x = layers.Activation("relu", name = f"{prefix}_act2")(x)
    x = layers.Dropout(0.2, name = f"{prefix}_dropout2")(x)
    return (x,y)
    
ecg_input_layer = tf.keras.layers.Input(shape=(5000,12), name="ecg_input")
age_input = tf.keras.layers.Input(shape=(1,), name="age_input")
x = layers.Conv1D(64, 16, padding='same', use_bias=False, name = "conv_1")(ecg_input_layer)
x = layers.BatchNormalization(name="bn")(x)
x = layers.Activation("relu", name="relu")(x)

x, y = residual_unit(x,x,1024,128, "res1")
x, y = residual_unit(x,y,256,196, "res2")
x, y = residual_unit(x,y,64,256, "res3")
x, _ = residual_unit(x,y,16,320, "res4")

x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Add(name="add")([x, age_input])
x = tf.keras.layers.Dense(1, name="output")(x)

model = tf.keras.models.Model([ecg_input_layer, age_input], x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 ecg_input (InputLayer)         [(None, 5000, 12)]   0           []                               
                                                                                                  
 conv_1 (Conv1D)                (None, 5000, 64)     12288       ['ecg_input[0][0]']              
                                                                                                  
 bn (BatchNormalization)        (None, 5000, 64)     256         ['conv_1[0][0]']                 
                                                                                                  
 relu (Activation)              (None, 5000, 64)     0           ['bn[0][0]']                     
                                                                                              

In [5]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

model_name = "resnet-age2"
output_path = make_checkpoint_dir("data/models", model_name)

print(f"Model: {model_name} Run Path: {output_path}")

In [9]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(output_path, save_best=True),
    tf.keras.callbacks.CSVLogger(f"data/models/{model_name}-history.csv")
]

history = model.fit(train_dataset, epochs=50, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/50


2023-11-27 10:08:07.138870: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2023-11-27 10:08:11.828534: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


   7479/Unknown - 312s 40ms/step - loss: 134.9000 - mse: 134.9000 - mae: 9.1246

INFO:tensorflow:Assets written to: data/models/resnet-age2-2023-11-27_10-08-01/assets


INFO:tensorflow:Assets written to: data/models/resnet-age2-2023-11-27_10-08-01/assets


...

7479/7479 [==============================] - 328s 44ms/step - loss: 100.6576 - mse: 100.6576 - mae: 7.8964 - val_loss: 123.8080 - val_mse: 123.8080 - val_mae: 8.7897 - lr: 0.0010
Epoch 5/50
7478/7479 [============================>.] - ETA: 0s - loss: 96.3943 - mse: 96.3943 - mae: 7.7214

INFO:tensorflow:Assets written to: data/models/resnet-age2-2023-11-27_10-08-01/assets


INFO:tensorflow:Assets written to: data/models/resnet-age2-2023-11-27_10-08-01/assets


7479/7479 [==============================] - 329s 44ms/step - loss: 96.3915 - mse: 96.3915 - mae: 7.7212 - val_loss: 188.8058 - val_mse: 188.8058 - val_mae: 11.1335 - lr: 0.0010
Epoch 6/50
4284/7479 [================>.............] - ETA: 2:06 - loss: 93.0601 - mse: 93.0601 - mae: 7.6056

KeyboardInterrupt: 

In [ ]:
model.save(f"data/models/{model_name}.keras")